In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

import sqlite3


In [ ]:
dir = '../input/sf-bay-area-bike-share/'
file_path = 'database.sqlite'
database = os.path.join(dir, file_path)
database 

In [ ]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""
SELECT *
FROM sqlite_master
WHERE type='table'
;""", conn)

tables

In [ ]:
trip = pd.read_sql("""
SELECT *
FROM trip
LIMIT 5
;""", conn)

trip

* How Many Stations Are Per City?

In [92]:
station_per_city = pd.read_sql("""
SELECT DISTINCT city, COUNT(name) Stations
FROM station
GROUP BY city
;""", conn)

station_per_city

,city,Stations
0,Mountain View,7
1,Palo Alto,5
2,Redwood City,7
3,San Francisco,35
4,San Jose,16


In [90]:
trips_per_station = pd.read_sql("""
SELECT 
    s.name Station_name,
    COUNT(t.start_station_id) Starting_trips,
    COUNT(t.end_station_id) Ending_trips
FROM station s
JOIN trip t
ON s.id = t.start_station_id
AND s.id = t.end_station_id
GROUP BY 1
ORDER BY 2 DESC
;""", conn)

trips_per_station

* How Many Trips Have Been Made From Each Station?

In [52]:
trips_from_station = pd.read_sql("""
SELECT 
    s.name Station_name,
    COUNT(t.start_station_id) Starting_trips
FROM station s
JOIN trip t
ON s.id = t.start_station_id
GROUP BY 1
ORDER BY 2 DESC
;""", conn)

trips_from_station

,Station_name,Starting_trips
0,San Francisco Caltrain (Townsend at 4th),49092
1,San Francisco Caltrain 2 (330 Townsend),33742
2,Harry Bridges Plaza (Ferry Building),32934
3,Embarcadero at Sansome,27713
4,Temporary Transbay Terminal (Howard at Beale),26089
...,...,...
65,Mezes Park,341
66,Redwood City Medical Center,311
67,San Mateo County Center,287
68,Franklin at Maple,224


* How Many Trips Have Been Made To Each Station?

In [88]:
trip_to_station = pd.read_sql("""
SELECT 
    s.name Station_Name,
    COUNT(t.end_station_id) Starting_trips
FROM station s
INNER JOIN trip t
ON s.id = t.end_station_id
GROUP BY 1
ORDER BY 2 DESC
;""", conn)

trip_to_station

,Station_Name,Starting_trips
0,San Francisco Caltrain (Townsend at 4th),63179
1,San Francisco Caltrain 2 (330 Townsend),35117
2,Harry Bridges Plaza (Ferry Building),33193
3,Embarcadero at Sansome,30796
4,2nd at Townsend,28529
...,...,...
65,Redwood City Medical Center,452
66,San Mateo County Center,350
67,Mezes Park,285
68,Redwood City Public Library,277


* How Many Subs/Cus. Are Per Station?
* How Many Trips Are Per Station?
* How Much Percentage Does Subs/Cus. Represent In Each Station ?

In [87]:
subs = pd.read_sql("""
SELECT 
    start_station_name,
    COUNT(CASE WHEN subscription_type='Subscriber' THEN start_station_id END) Subscribers,
    COUNT(CASE WHEN subscription_type='Customer' THEN start_station_id END) Customers,
    (SELECT COUNT(start_station_id) FROM trip t1 WHERE t.start_station_id = t1.start_station_id) Total_trips,   
    COUNT(CASE WHEN subscription_type='Subscriber' THEN start_station_id END)*100/COUNT(start_station_id) Sub_percentage,
    COUNT(CASE WHEN subscription_type='Customer' THEN start_station_id END)*100/COUNT(start_station_id) Cus_percentage
FROM trip t
GROUP BY 1
;""", conn)

subs

,start_station_name,Subscribers,Customers,Total_trips,Sub_percentage,Cus_percentage
0,2nd at Folsom,14732,1208,15940,92,7
1,2nd at South Park,16945,1551,18496,91,8
2,2nd at Townsend,22482,3355,25837,87,12
3,5th at Howard,11631,1895,13526,85,14
4,Adobe on Almaden,1005,252,1257,79,20
...,...,...,...,...,...,...
69,Townsend at 7th,22118,1606,23724,93,6
70,University and Emerson,385,1127,1512,25,74
71,Washington at Kearney,911,561,5764,61,38
72,Washington at Kearny,2801,1491,5764,65,34
